In [1]:
!pip install langchain
!pip install langchain_openai

In [1]:
import yfinance as yf
data = yf.download("AAPL", start="2010-01-01", end="2023-01-01")
#returns = data.pct_change().dropna()
print (data)

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed

Price            Close        High         Low        Open     Volume
Ticker            AAPL        AAPL        AAPL        AAPL       AAPL
Date                                                                 
2010-01-04    6.440331    6.455077    6.391279    6.422877  493729600
2010-01-05    6.451464    6.487877    6.417458    6.458085  601904800
2010-01-06    6.348846    6.477045    6.342226    6.451466  552160000
2010-01-07    6.337110    6.379844    6.291067    6.372320  477131200
2010-01-08    6.379240    6.379842    6.291368    6.328683  447610800
...                ...         ...         ...         ...        ...
2022-12-23  130.344513  130.898074  128.150027  129.415314   63814900
2022-12-27  128.535492  129.899636  127.240551  129.869982   69007800
2022-12-28  124.591385  129.524031  124.423341  128.179661   85438400
2022-12-29  128.120346  128.980342  126.261956  126.518963   75703700
2022-12-30  128.436661  128.456435  125.965402  126.934142   77034200

[3272 rows x 5 colu

In [6]:
%%writefile black_litterman.py

import numpy as np
import pandas as pd
import yfinance as yf
from scipy.optimize import minimize
import matplotlib.pyplot as plt
import requests


def get_user_input():
    num_assets = 2 #int(input("Enter the number of stocks: "))
    stock_symbols = []

    stock_symbols.append("AAPL")
    stock_symbols.append("MSFT")

    time_horizon = 0.019*2
    return stock_symbols, time_horizon

def fetch_historical_data(stock_symbols, start_date="2010-01-01", end_date="2023-01-01"):
    data = yf.download(stock_symbols, start=start_date, end=end_date)['Close']
    returns = data.pct_change().dropna()
    return returns

def calculate_expected_returns_and_cov_matrix(returns, time_horizon):
    expected_returns = returns.mean() * time_horizon
    cov_matrix = returns.cov() * time_horizon
    return expected_returns, cov_matrix

def calculate_market_equilibrium_returns(cov_matrix, market_caps):
    market_weights = market_caps / np.sum(market_caps)
    equilibrium_returns = np.dot(cov_matrix, market_weights)
    return equilibrium_returns

def adjust_returns_with_views(equilibrium_returns, cov_matrix, P, Q, omega):
    tau = 0.025
    M_inverse = np.linalg.inv(np.dot(np.dot(tau, P), np.dot(cov_matrix, P.T)) + omega)
    adjusted_returns = equilibrium_returns + np.dot(np.dot(np.dot(cov_matrix, P.T), M_inverse), (Q - np.dot(P, equilibrium_returns)))
    return adjusted_returns

def portfolio_variance(weights, cov_matrix):
    return np.dot(weights.T, np.dot(cov_matrix, weights))

def portfolio_return(weights, expected_returns):
    return np.dot(weights, expected_returns)

def optimize_portfolio(expected_returns, cov_matrix, target_return):
    num_assets = len(expected_returns)

    def constraint(weights):
        return np.sum(weights) - 1

    def return_constraint(weights):
        return portfolio_return(weights, expected_returns) - target_return

    bounds = [(0, 1) for _ in range(num_assets)]
    initial_weights = [1. / num_assets] * num_assets

    result = minimize(portfolio_variance, initial_weights, args=(cov_matrix,), method='SLSQP', bounds=bounds, constraints=[{'type': 'eq', 'fun': constraint}, {'type': 'eq', 'fun': return_constraint}])

    return result.x

def calculate_efficient_frontier(expected_returns, cov_matrix, num_portfolios=100):
    target_returns = np.linspace(min(expected_returns), max(expected_returns), num_portfolios)
    frontier_returns = []
    frontier_risks = []
    frontier_weights = []

    for target_return in target_returns:
        weights = optimize_portfolio(expected_returns, cov_matrix, target_return)
        frontier_returns.append(portfolio_return(weights, expected_returns))
        frontier_risks.append(np.sqrt(portfolio_variance(weights, cov_matrix)))
        frontier_weights.append(weights)

    return frontier_returns, frontier_risks, frontier_weights

def plot_efficient_frontier(frontier_returns, frontier_risks, frontier_weights, risk_free_rate=0.0106):
    '''
    plt.figure(figsize=(10, 6))
    plt.scatter(frontier_risks, frontier_returns, c=np.array(frontier_returns)/np.array(frontier_risks), cmap='YlGnBu', marker='o')
    plt.title('Efficient Frontier')
    plt.xlabel('Risk (Standard Deviation)')
    plt.ylabel('Return')
    plt.colorbar(label='Sharpe Ratio')
    '''
    max_sharpe_idx = np.argmax(np.array(frontier_returns) / np.array(frontier_risks))
    max_sharpe_ratio = frontier_returns[max_sharpe_idx] / frontier_risks[max_sharpe_idx]
    max_sharpe_return = frontier_returns[max_sharpe_idx]
    max_sharpe_risk = frontier_risks[max_sharpe_idx]

    #plt.scatter(max_sharpe_risk, max_sharpe_return, marker='*', color='r', s=100, label='Max Sharpe Ratio')

    cml_x = [0, max_sharpe_risk]
    cml_y = [risk_free_rate, max_sharpe_return]
    '''
    plt.plot(cml_x, cml_y, color='r', linestyle='--', label='Capital Market Line (CML)')

    plt.legend()
    #plt.show()
    '''

    #print("Optimal Weights for Maximum Sharpe Ratio Portfolio:")
    weights = []
    for i in range(len(frontier_weights[max_sharpe_idx])):
        #print(f"Stock {i+1}: {frontier_weights[max_sharpe_idx][i]:.4f}")
        formatted_value = round(frontier_weights[max_sharpe_idx][i], 4)
        weights.append(formatted_value)
    return weights

def blackLitterman(p_value, q_value):
    stock_symbols, time_horizon = get_user_input()

    returns = fetch_historical_data(stock_symbols)
    expected_returns, cov_matrix = calculate_expected_returns_and_cov_matrix(returns, time_horizon)

    # Fetch market capitalizations
    market_caps = []
    for symbol in stock_symbols:
        ticker = yf.Ticker(symbol)
        market_cap = ticker.info['marketCap']
        market_caps.append(market_cap)

    equilibrium_returns = calculate_market_equilibrium_returns(cov_matrix, market_caps)

    # Define P based on p_value
    if p_value == 0:
        P = np.array([[1, -1]])
    elif p_value == 1:
        P = np.array([[-1, 1]])
    elif p_value == 2:
        P = np.array([[1, 0]])
    elif p_value == 3:
        P = np.array([[0, 1]])
    else:
        raise ValueError("Invalid p_value. Must be 0, 1, 2, or 3.")

    Q = np.array([q_value])

    # Set uncertainty in views (omega) as a diagonal matrix with small values
    omega = np.diag([0.0001])

    adjusted_returns = adjust_returns_with_views(equilibrium_returns, cov_matrix, P, Q, omega)

    #print("Expected Returns:", adjusted_returns)
    #print("Covariance Matrix:\n", cov_matrix)

    frontier_returns, frontier_risks, frontier_weights = calculate_efficient_frontier(adjusted_returns, cov_matrix)

    return plot_efficient_frontier(frontier_returns, frontier_risks, frontier_weights)


Overwriting black_litterman.py


In [7]:
%%writefile app.py



jobdescription = "You are a portfolio manager and are looking at the changes to a portfolio made from anlysing the sentiment of current news aricles."
message_1 ="Hello there! Why have I lost money?"

import sys
# Append the directory to your python path using sys
sys.path.append('/content/drive/MyDrive')
# Import the module

from black_litterman import *

import torch
import torch.nn as nn
import torch.optim as optim
from torch.distributions import Categorical
from transformers import BertModel, BertTokenizer
import csv
import pandas as pd
from openai import OpenAI
import streamlit as st
import numpy as np

API_KEY = "02631291-6a1d-4f18-847d-ad4d8dcce934"
# Hyperparameters
EMBEDDING_DIM = 768  # BERT embedding dimension
HIDDEN_DIM = 256
ACTION_SPACE = 5
LEARNING_RATE = 1e-4
GAMMA = 0.99  # Discount factor for rewards
BATCH_SIZE = 10  # Number of episodes before updating the policy

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Transformer-based Policy Network
import torch
import torch.nn as nn
from transformers import BertModel

class TransformerPolicy(nn.Module):
    def __init__(self, action_space, hidden_dim=1024, num_layers=24, num_heads=16, use_residual=True, dropout_rate=0.1):
        super(TransformerPolicy, self).__init__()
        # Load pre-trained BERT model
        model_dir = os.path.expanduser("~/.cache/huggingface/hub/models--bert-base-uncased")
        if os.path.exists(model_dir):
            self.bert = BertModel.from_pretrained('bert-base-uncased', local_files_only=True)
        else:
            self.bert = BertModel.from_pretrained('bert-base-uncased')
        #self.bert = BertModel.from_pretrained('bert-base-uncased', local_files_only=True)

        # Define a much deeper and more complex network
        self.use_residual = use_residual
        self.layers = nn.ModuleList()
        input_dim = self.bert.config.hidden_size  # BERT's hidden size (768 for bert-base-uncased)

        # Initial projection to hidden_dim
        self.projection = nn.Linear(input_dim, hidden_dim)

        # Multi-head self-attention blocks
        self.attention_blocks = nn.ModuleList()
        for _ in range(num_layers // 2):  # Add attention blocks every 2 layers
            self.attention_blocks.append(
                nn.MultiheadAttention(embed_dim=hidden_dim, num_heads=num_heads, dropout=dropout_rate)
            )

        # Deep feed-forward network
        for i in range(num_layers):
            # Add a linear layer with intermediate expansion
            self.layers.append(nn.Sequential(
                nn.Linear(hidden_dim, hidden_dim * 4),  # Expand dimension
                nn.GELU(),
                nn.Linear(hidden_dim * 4, hidden_dim),  # Project back to hidden_dim
                nn.Dropout(dropout_rate)
            ))
            # Add Layer Normalization (pre-norm)
            self.layers.append(nn.LayerNorm(hidden_dim))

        # Final layer to map to action space
        self.final_layer = nn.Linear(hidden_dim, action_space)
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, input_ids, attention_mask):
        # Get BERT embeddings
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output  # Use the [CLS] token representation

        # Project BERT output to hidden_dim
        x = self.projection(pooled_output)

        # Pass through the deep network with attention blocks
        attention_block_idx = 0
        for i, layer in enumerate(self.layers):
            if isinstance(layer, nn.Sequential):  # Feed-forward block
                residual = x if self.use_residual else 0
                x = layer(x)
                if self.use_residual:
                    x = x + residual
            elif isinstance(layer, nn.LayerNorm):  # Layer normalization
                x = layer(x)

            # Insert multi-head self-attention every 2 layers
            if i % 2 == 0 and attention_block_idx < len(self.attention_blocks):
                x = x.unsqueeze(0)  # Add batch dimension for attention
                x, _ = self.attention_blocks[attention_block_idx](x, x, x)
                x = x.squeeze(0)  # Remove batch dimension
                attention_block_idx += 1

        # Final layer to produce logits
        logits = self.final_layer(x)
        logits = torch.clamp(logits, min=-10, max=10)  # Clip logits to avoid extreme values
        action_probs = self.softmax(logits + 1e-9)  # Add epsilon for numerical stability
        return action_probs


# Reinforcement Learning Agent
class RLAgent:
    def __init__(self, policy_net, optimizer, gamma):
        self.policy_net = policy_net
        self.optimizer = optimizer
        self.gamma = gamma
        self.saved_log_probs = []
        self.rewards = []


    def select_action(self, state):
        input_ids = state['input_ids'].to(device)
        attention_mask = state['attention_mask'].to(device)

        # Get action probabilities
        action_probs = self.policy_net(input_ids, attention_mask)

        # Clone before modification to avoid in-place changes
        modified_probs = action_probs.clone()

        # Define the excluded action
        excluded_action = 0

        # Store the probability of the excluded action
        excluded_action_prob = modified_probs[0, excluded_action].item()

        # Set the probability of the excluded action to 0
        modified_probs[0, excluded_action] = 0.0

        # Normalize the remaining probabilities
        modified_probs = modified_probs / modified_probs.sum()

        # Sample an action from the modified probability distribution
        m = Categorical(modified_probs)
        action = m.sample()

        # Save log probability for training
        self.saved_log_probs.append(m.log_prob(action))

        return action.item(), excluded_action_prob




    def update_policy(self):
        R = 0
        policy_loss = []
        returns = []

        # Calculate discounted returns
        for r in self.rewards[::-1]:
            R = r + self.gamma * R
            returns.insert(0, R)

        returns = torch.tensor(returns).to(device)
        if len(returns) > 1:  # Only normalize if there are multiple returns
            returns = (returns - returns.mean()) / (returns.std() + 1e-9)  # Normalize returns

        # Calculate policy loss
        for log_prob, R in zip(self.saved_log_probs, returns):
            policy_loss.append(-log_prob * R)

        # Optimize the policy
        self.optimizer.zero_grad()
        policy_loss = torch.cat(policy_loss).sum()
        policy_loss.backward()
        self.optimizer.step()

        # Clear saved rewards and log probabilities
        self.saved_log_probs = []
        self.rewards = []

# Tokenizer for text input
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
@st.cache_resource
def load_transformer_policy():
    return TransformerPolicy(ACTION_SPACE).to(device)


# Initialize policy network and optimizer
policy_net = load_transformer_policy()
optimizer = optim.Adam(policy_net.parameters(), lr=LEARNING_RATE)
agent = RLAgent(policy_net, optimizer, GAMMA)

def load_data():
  df1 = pd.read_csv('AppleNewsStock.csv')
  df2 = pd.read_csv('MicrosoftNewsStock.csv')
  return df1, df2

def process_data(df1, df2):
  processed_data = []
  days = 7
  data_chunk = ""
  for (index1, row1), (index2, row2) in zip(df1.iterrows(), df2.iterrows()):
    if days>0:
      data_chunk += str(row1['Open'])+str(row1['High'])+str(row1['Low'])+str(row1['Close'])+str(row1['Adj Close'])+str(row1['Volume'])+str(row1['News'])
      data_chunk += str(row2['Open'])+str(row2['High'])+str(row2['Low'])+str(row2['Close'])+str(row2['Adj Close'])+str(row2['Volume'])+str(row2['News'])
      days-=1
    else:
      days = 7
      processed_data.append(data_chunk)
      data_chunk = ""
  return processed_data

def get_news_for_time_period(timestamp):
    """Retrieve news articles corresponding to a given time period."""
    df1,df2 = load_data()
    filtered_df = df1[df1['Date'] == timestamp]
    if not filtered_df.empty:
        return filtered_df.iloc[0]['News']
    return None
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage

class AI(ChatOpenAI):
    def __init__(self, model_name="Llama-3.3-70B-Instruct",
                 openai_api_key="02631291-6a1d-4f18-847d-ad4d8dcce934",
                 openai_api_base="https://api.arliai.com/v1",
                 temperature=0.8):

        # Call the parent constructor with valid arguments
        super().__init__(model_name=model_name, openai_api_key=openai_api_key,openai_api_base="https://api.arliai.com/v1", temperature=temperature)
def generate_chatgpt_explanation(p, q, timestamp):
    news_articles = get_news_for_time_period(timestamp)
    if p == 0:
        P = np.array([[1, -1]])
    elif p == 1:
        P = np.array([[-1, 1]])
    elif p == 2:
        P = np.array([[1, 0]])
    elif p == 3:
        P = np.array([[0, 1]])

    """Generate an explanation for why P and Q values were chosen based on news context."""
    prompt = f""" The 0th index in the p value is the apple's stock and the 1st index being microsoft stock.
    p values can be of 4 types, [1,-1] which means the price of 0th index is going to outperform the price of 1st index by the amount given in the Q value, [-1,1] means 1st index is going to outperform 2nd index by it's q value,[1,0] means price of 0th index is going to rise by its q value and [0,1] price of 1st index is going to rise by its q value
    The following are the P and Q values predicted by a reinforcement learning model:
    - P value: {p}
    - Q value: {q}

    The corresponding news articles for this time period are:
    {news_articles}

    Based on the news context, explain why these P and Q values were chosen by the model.
    """

    #return response["choices"][0]["message"]["content"]
    llm = AI()

    messages = [
    SystemMessage(content="You are a experienced stock portfolio manager."),
    HumanMessage(content= prompt)]

    # Generate response
    response = llm.invoke(messages)


    return(response.content)


# Example training loop
def train(episodes):
    df1, df2 = load_data()
    processed_data = process_data(df1, df2)
    #return response["choices"][0]["message"]["content"]

def main():
    x = []
    y = []
    st.title("AI Based Black Litterman Portfolio Optimisation")
    image_url = "https://easydrawingguides.com/wp-content/uploads/2017/02/How-to-draw-a-cartoon-tree-20.png"
    image_placeholder = st.empty()

    # Load and process data
    df1, df2 = load_data()
    processed_data = process_data(df1, df2)
    
    # Initialize policy network and RL agent
    policy_net = load_transformer_policy()
    optimizer = optim.Adam(policy_net.parameters(), lr=LEARNING_RATE)
    agent = RLAgent(policy_net, optimizer, GAMMA)

    episodes = 300-240
    balance=10000

    for episode in range(episodes):
      if episode%7==0:
        text = processed_data[episode]
        inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True)

        # Select action
        action, excluded_action_prob = agent.select_action(inputs)
        explanation = generate_chatgpt_explanation(p = action, q = excluded_action_prob,timestamp = df1.iloc[episode].loc['Date'])
        st.write(f"Episode {episode + 1}: Selected Action: {action}, Q-value: {excluded_action_prob}")
        aapl, msft= blackLitterman(action-1, excluded_action_prob)


        st.write(f"The weights for the first week's optimised portfolio: Apple: {aapl}, Microsoft: {msft}")

        aapl_in_pounds = balance*aapl
        msft_in_pounds = balance*msft

        st.write(f"Apple: {df1.iloc[episode].loc['Date']}")
        st.write(f"Microsoft: {df2.iloc[episode].loc['Date']}")

        aapl_shares = aapl_in_pounds/df1.iloc[episode].loc['Open']
        msft_shares = msft_in_pounds/df2.iloc[episode].loc['Open']



        aapl_sell_price = aapl_shares * (df1.iloc[episode+3].loc['Open'])
        msft_sell_price = msft_shares * (df2.iloc[episode+3].loc['Open'])

        if "show_image" not in st.session_state:
          st.session_state["show_image"] = True

        if st.session_state["show_image"]:
          image_placeholder.image(image_url, use_column_width=True)
        else:
          image_placeholder.empty()

        aapl_profit = aapl_sell_price-aapl_in_pounds
        msft_profit = msft_sell_price-msft_in_pounds
        total_profit = msft_profit+aapl_profit
        if total_profit<0:
          total_profit*=-1
        st.write(f"Total Profit: {total_profit}")
    # Streamlit Sidebar UI
    with st.sidebar:
        st.write("AI-based Stock Portfolio Optimization")

        # Sample explanation generation
        timestamp = df1.iloc[0]['Date']  # Example: Taking the first date from dataset
        p, q = 2, 0.05  # Example P and Q values
        st.write("AI Explanation for P and Q Values:")
        st.write(explanation)


    st.write("Portfolio Optimization Completed!")

if __name__ == "__main__":
    main()


Overwriting app.py


In [ ]:
!streamlit run app.py